In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END


class State(TypedDict):
  topic: str
  joke: str


def refine_topic(state: State):
    return {"topic": state["topic"] + " and cats"}


def generate_joke(state: State):
    return {"joke": f"This is a joke about {state['topic']}"}

graph = (
  StateGraph(State)
  .add_node(refine_topic)
  .add_node(generate_joke)
  .add_edge(START, "refine_topic")
  .add_edge("refine_topic", "generate_joke")
  .add_edge("generate_joke", END)
  .compile()
)

In [3]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode="updates",  
):
    print(chunk)

{'refine_topic': {'topic': 'ice cream and cats'}}
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


In [4]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode="values",  
):
    print(chunk)

{'topic': 'ice cream'}
{'topic': 'ice cream and cats'}
{'topic': 'ice cream and cats', 'joke': 'This is a joke about ice cream and cats'}


In [5]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode=["updates", "values"],  
):
    print(chunk)

('values', {'topic': 'ice cream'})
('updates', {'refine_topic': {'topic': 'ice cream and cats'}})
('values', {'topic': 'ice cream and cats'})
('updates', {'generate_joke': {'joke': 'This is a joke about ice cream and cats'}})
('values', {'topic': 'ice cream and cats', 'joke': 'This is a joke about ice cream and cats'})
